In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm

from IPython.display import clear_output

# Methods

In [2]:
class LinearRegression:

    @staticmethod
    def get_polynomial_basis(X, degree):
        def partition(n, k):
            if k==1:
                yield np.array([n])
            else:
                for i in range(n+1):
                    for result in partition(n-i, k-1):
                        yield np.concatenate(([i], result))

        basis = []
        for m in range(degree+1):
            for p in partition(degree-m, X.shape[1]):
                basis.append(np.prod(X**p, axis=1))
        return np.vstack(basis).T
        
    @staticmethod
    def get_gaussian_basis(X, n_clusters, sigma):
        def k_means(X, k):
            n_samples = X.shape[0]
            means = X[np.random.choice(n_samples, k, replace=False)]
            labels, labels_old = np.zeros(n_samples), np.zeros(n_samples)
            while True:
                for i in range(n_samples):
                    labels[i] = np.argmin(np.sum((X[i]-means)**2, axis=-1))
                for j in range(k):
                    means[j] = np.mean(X[labels==j], axis=0)
                if np.all(labels==labels_old):
                    break
                labels_old = labels.copy()
            return means
        
        n_samples = X.shape[0]
        kmeans = k_means(X, n_clusters-1)
        basis = np.zeros((X.shape[0], n_clusters))
        basis[:, 0] = 1
        for i in range(1, n_clusters):
            means_ = np.expand_dims(kmeans[i-1], axis=0).repeat(n_samples, axis=0)
            basis[:, i] = np.exp( np.sum((X-means_)**2, axis=1) / (-2*sigma**2))
        return basis
    
    @staticmethod
    def get_weights(X, y, lambda_=0):
        return np.linalg.inv(X.T @ X + lambda_*np.eye(X.shape[1])) @ X.T @ y


In [3]:
class BuildDataset:
    def __init__(self, dataset, size):
        self.dataset = dataset
        self.size = size

        train = pd.read_csv(f"{dataset}/train_1_{size}.csv")
        val = pd.read_csv(f"{dataset}/val_1.csv")
        test = pd.read_csv(f"{dataset}/test_1.csv")

        self.X_train = train.iloc[:, :-1].values.astype(np.float32)
        self.y_train = train.iloc[:, -1].values.astype(np.float32)
        self.X_val = val.iloc[:, :-1].values.astype(np.float32)
        self.y_val = val.iloc[:, -1].values.astype(np.float32)
        self.X_test = test.iloc[:, :-1].values.astype(np.float32)
        self.y_test = test.iloc[:, -1].values.astype(np.float32)

    def fit(self, model_name, lambdas, degree=1, n_clusters=1, sigma=1):
        if model_name == "linear":
            self.basis = lambda X: X
        elif model_name == "polynomial":
            self.degree = degree
            self.basis = lambda X: LinearRegression.get_polynomial_basis(X, degree)
        elif model_name == "gaussian":
            self.n_clusters = n_clusters
            self.sigma = sigma
            self.basis = lambda X: LinearRegression.get_gaussian_basis(X, n_clusters, sigma)

        X_train_basis = self.basis(self.X_train)
        X_val_basis = self.basis(self.X_val)
        X_test_basis = self.basis(self.X_test)
        self.w = {}
        self.train_loss, self.val_loss, self.test_loss = {}, {}, {}
        for lambda_ in lambdas:

            self.w[lambda_] = LinearRegression.get_weights(X_train_basis, self.y_train, lambda_)

            self.train_loss[lambda_] = np.mean((X_train_basis @ self.w[lambda_] - self.y_train)**2)
            self.val_loss[lambda_] = np.mean((X_val_basis @ self.w[lambda_] - self.y_val)**2)
            self.test_loss[lambda_] = np.mean((X_test_basis @ self.w[lambda_] - self.y_test)**2)
        
        self.best_lambda = min(self.val_loss, key=self.val_loss.get)

# Tasks

In [ ]:
# Task 1
def plot_approximated_functions(dataset, mode='Train'):
    if mode=='Train':
        X = dataset.X_train
        y = dataset.y_train
    elif mode=='Val':
        X = dataset.X_val
        y = dataset.y_val
    elif mode=='Test':
        X = dataset.X_test
        y = dataset.y_test

    for lambda_ in dataset.w.keys():
        x_plot = np.sort(X[:, 0]).reshape(-1, 1)

        fig, ax = plt.subplots(figsize=(8, 6))

        ax.plot(x_plot, y, 'o', markersize=8, label='Data', color='limegreen', alpha=0.8)
        ax.plot(x_plot, dataset.basis(x_plot) @ dataset.w[lambda_], label='Model', color='blue', linewidth=2.5)

        ax.set_xlabel('x', fontsize=14, color='black')
        ax.set_ylabel('y', fontsize=14, color='black')
        
        # ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        legend = ax.legend()
        frame = legend.get_frame()
        frame.set_edgecolor('black')

        plt.style.use('default')

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_tick_params(width=1.5, color='black')
        ax.yaxis.set_tick_params(width=1.5, color='black')

        ax.tick_params(axis='both', colors='black')
        ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        plt.tight_layout()

        plt.savefig(f"assets/task1_{dataset.dataset}_{dataset.size}_{mode}_degree{dataset.degree}_lambda{lambda_}.png", dpi=100)

        plt.show()


In [ ]:
# Dataset 1 | Task 1

sizes = [10]
degrees = [4]
lambdas = [0]

for size in sizes:
    for degree in degrees:
        dataset_1 = BuildDataset("dataset_1", size)
        dataset_1.fit("polynomial", degree=degree, lambdas=lambdas)
        # plot_approximated_functions(dataset_1, mode='Train')
        # plot_approximated_functions(dataset_1, mode='Val')
        # plot_approximated_functions(dataset_1, mode='Test')
clear_output()

---

In [ ]:
# Task 2
def plot_approximated_surface(dataset, mode='Train'):
    if mode=='Train':
        X = dataset.X_train
        y = dataset.y_train
    elif mode=='Val':
        X = dataset.X_val
        y = dataset.y_val
    elif mode=='Test':
        X = dataset.X_test
        y = dataset.y_test

    for lambda_ in dataset.w.keys():
        # 2d plot
        x1_plot = np.sort(X[:, 0]).reshape(-1, 1)
        x2_plot = np.sort(X[:, 1]).reshape(-1, 1)
        xx1, xx2 = np.meshgrid(x1_plot, x2_plot)
        xx = np.hstack((xx1.reshape(-1, 1), xx2.reshape(-1, 1)))
        yy = dataset.basis(xx) @ dataset.w[lambda_]
        
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(projection='3d')

        cmap = cm.get_cmap('plasma')

        surf = ax.plot_surface(xx1, xx2, yy.reshape(xx1.shape), cmap=cmap, rstride=1, cstride=1, alpha=0.8, antialiased=True)
        # 3d scatter
        surf2 = ax.scatter(X[:, 0], X[:, 1], y, color='limegreen', marker='o', alpha=0.8)

        ax.set_xlabel('$x_1$', fontsize=14, color='black')
        ax.set_ylabel('$x_2$', fontsize=14, color='black')
        ax.set_zlabel('$y$', fontsize=14, color='black')
        # ax.set_title(f'{mode} ($\lambda=${lambda_})', fontsize=16)

        ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
        ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

        ax.view_init(elev=20, azim=-45)
        ax.tick_params(axis='both', colors='black')
        ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        plt.tight_layout()

        plt.savefig(f"assets/task2_{dataset.dataset}_{dataset.size}_{mode}_lambda{lambda_}.png", dpi=100) #_degree{dataset.degree}

        plt.show()

In [ ]:
# Dataset 2 | Task 2

sizes = [50, 200]
degrees = [2, 3, 6]
lambdas = [0, 1e-4, 0.1, 5, 100]

for size in sizes:
    for degree in degrees:
        dataset_2 = BuildDataset("dataset_2", size=size)
        dataset_2.fit("polynomial", degree=degree, lambdas=lambdas)

        # plot_approximated_surface(dataset_2, mode='Train')
        # plot_approximated_surface(dataset_2, mode='Val')
        # plot_approximated_surface(dataset_2, mode='Test')
clear_output()

---

In [11]:
# Task 3
def plot_scatter(dataset, mode='Test'):
    X_train = dataset.X_train
    y_train = dataset.y_train
    if mode=='Val':
        X = dataset.X_val
        y = dataset.y_val
    elif mode=='Test':
        X = dataset.X_test
        y = dataset.y_test

    for lambda_ in dataset.w.keys():
        # scatter plot between truth and predicted 
        fig, ax = plt.subplots(figsize=(8, 6))

        ax.scatter(y_train, dataset.basis(X_train) @ dataset.w[lambda_], label='Train', color='limegreen', alpha=0.8)
        ax.scatter(y, dataset.basis(X) @ dataset.w[lambda_], label=mode, color='blue', alpha=0.8)

        ax.set_xlabel('Truth', fontsize=14, color='black')
        ax.set_ylabel('Predicted', fontsize=14, color='black')
        
        # ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        legend = ax.legend()
        frame = legend.get_frame()
        frame.set_edgecolor('black')

        plt.style.use('default')

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_tick_params(width=1.5, color='black')
        ax.yaxis.set_tick_params(width=1.5, color='black')

        ax.tick_params(axis='both', colors='black')
        ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        plt.tight_layout()

        plt.savefig(f"assets/task3_{dataset.dataset}_{dataset.size}_{mode}_degree{dataset.degree}_lambda{lambda_}.png", dpi=100)

        plt.show()

In [16]:
# Dataset 2 | Task 3 

sizes = [50, 200]
degrees = [6]
lambdas = [0, 1e-4, 0.1, 5, 100]

for size in sizes:
    for degree in degrees:
        dataset_2 = BuildDataset("dataset_2", size=size)
        dataset_2.fit("polynomial", degree=degree, lambdas=lambdas)

        plot_scatter(dataset_2, mode='Val')
        plot_scatter(dataset_2, mode='Test')
clear_output()

In [ ]:
# Task 3
def plot_scatter(dataset, sigma, mode='Test'):
    X_train = dataset.X_train
    y_train = dataset.y_train
    if mode=='Val':
        X = dataset.X_val
        y = dataset.y_val
    elif mode=='Test':
        X = dataset.X_test
        y = dataset.y_test

    for lambda_ in dataset.w.keys():
        # scatter plot between truth and predicted 
        fig, ax = plt.subplots(figsize=(8, 6))

        ax.scatter(y_train, dataset.basis(X_train) @ dataset.w[lambda_], label='Train', color='limegreen', alpha=0.8)
        ax.scatter(y, dataset.basis(X) @ dataset.w[lambda_], label=mode, color='blue', alpha=0.8)

        ax.set_xlabel('Truth', fontsize=14, color='black')
        ax.set_ylabel('Predicted', fontsize=14, color='black')
        
        # ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        legend = ax.legend()
        frame = legend.get_frame()
        frame.set_edgecolor('black')

        plt.style.use('default')

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_tick_params(width=1.5, color='black')
        ax.yaxis.set_tick_params(width=1.5, color='black')

        ax.tick_params(axis='both', colors='black')
        ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        plt.tight_layout()

        plt.savefig(f"assets/task3_{dataset.dataset}_{dataset.size}_{mode}_cluster{dataset.n_clusters}_sigma_{sigma}_lambda{lambda_}.png", dpi=100)

        plt.show()

In [ ]:
# Dataset 2 | Task 3 

sizes = [50, 200]
n_clusterss = [25, 30, 45]
sigmas = [1]
lambdas = [0, 1e-4, 0.1, 5, 100]

for size in sizes:
    for sigma in sigmas:
        for n_clusters in n_clusterss:
            dataset_2 = BuildDataset("dataset_2", size=size)
            dataset_2.fit("gaussian", n_clusters=n_clusters, sigma=sigma, lambdas=lambdas)

            # plot_scatter(dataset_2, sigma, mode='Val')
            # plot_scatter(dataset_2, sigma, mode='Test')
clear_output()

In [ ]:
# Dataset 3 | Task 3 

sizes = [361]
n_clusterss = [2]
sigmas = [20, 25, 30]
lambdas = [0, 1e-4, 0.1, 5, 100]

for size in sizes:
    for sigma in sigmas:
        for n_clusters in n_clusterss:
            dataset_3 = BuildDataset("dataset_3", size=size)
            dataset_3.fit("gaussian", n_clusters=n_clusters, sigma=sigma, lambdas=lambdas)

            # plot_scatter(dataset_3, sigma, mode='Val')
            # plot_scatter(dataset_3, sigma, mode='Test')
clear_output()

---

In [ ]:
# Task 3
def plot_scatter(dataset, mode='Test'):
    X_train = dataset.X_train
    y_train = dataset.y_train
    if mode=='Val':
        X = dataset.X_val
        y = dataset.y_val
    elif mode=='Test':
        X = dataset.X_test
        y = dataset.y_test

    for lambda_ in dataset.w.keys():
        # scatter plot between truth and predicted 
        fig, ax = plt.subplots(figsize=(8, 6))

        ax.scatter(y_train, dataset.basis(X_train) @ dataset.w[lambda_], label='Train', color='limegreen', alpha=0.8)
        ax.scatter(y, dataset.basis(X) @ dataset.w[lambda_], label=mode, color='blue', alpha=0.8)

        ax.set_xlabel('Truth', fontsize=14, color='black')
        ax.set_ylabel('Predicted', fontsize=14, color='black')
        
        # ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        legend = ax.legend()
        frame = legend.get_frame()
        frame.set_edgecolor('black')

        plt.style.use('default')

        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.xaxis.set_tick_params(width=1.5, color='black')
        ax.yaxis.set_tick_params(width=1.5, color='black')

        ax.tick_params(axis='both', colors='black')
        ax.grid(True, linestyle='--', alpha=0.5, color='gray')

        plt.tight_layout()

        plt.savefig(f"assets/task3_{dataset.dataset}_{dataset.size}_{mode}_degree{dataset.degree}_lambda{lambda_}.png", dpi=100)

        plt.show()

In [ ]:
# Dataset 3 | Task 3

sizes = [1024]
degrees = [1, 2, 4]
lambdas = [0, 1e-4, 0.1, 5, 100]

for size in sizes:
    for degree in degrees:
        dataset_4 = BuildDataset("dataset_4", size=size)
        dataset_4.fit("polynomial", degree=degree, lambdas=lambdas)

        # plot_scatter(dataset_4, mode='Val')
        # plot_scatter(dataset_4, mode='Test')
clear_output()